In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
#Objetos que dan formato al dataframe
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.types import Row
from pyspark.sql import SQLContext

#Mala práctica hacer Import *
import pyspark.sql.functions
from pyspark.sql.functions import count
from pyspark.sql import SparkSession

#Objetos para Particionar
from pyspark.storagelevel import StorageLevel

In [2]:
sc = SparkContext(master="local", appName="particionado")

In [3]:
#Se llama
sqlContext = SQLContext(sc)

A:\Instal_python\lib\site-packages\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
var = "C:/Users/Caroline/Videos/curso-apache-spark-platzi/files/"

depteRDD = sc.textFile(var+"deporte.csv").map(lambda line : line.split(","))

In [5]:
#Función para validar el encabezado
#Iter: Devuelve valor a valor lo que se está procesando
def deletHeader(indice, interador):
    return iter(list(interador)[1:])

In [6]:
#Eliminar el encabezado
depteRDD = depteRDD.mapPartitionsWithIndex(deletHeader) #Deporte

In [7]:
#Antes de Tramsformar los datos del RDD, se debe transformar el tipo de dato
#Deporte
depteRDD = depteRDD.map(lambda l: (
int(l[0]),
l[1]
))

#Se crear el Schema

#Deporte
depte = StructType([
    StructField("deporte_id", IntegerType(), False),
    StructField("nombre_deporte", StringType(), False)
])

In [8]:
#Aplicar el esquema al RDD

#Deporte
depteDF = sqlContext.createDataFrame(depteRDD, depte)

In [9]:
depteDF.show(5)

+----------+--------------+
|deporte_id|nombre_deporte|
+----------+--------------+
|         1|    Basketball|
|         2|          Judo|
|         3|      Football|
|         4|    Tug-Of-War|
|         5| Speed Skating|
+----------+--------------+
only showing top 5 rows



In [10]:
depteDF.is_cached

False

In [11]:
#Para almacenar el RDD en cache y reducir los tiempos de consulta,
#ya que al no estar almacenado Spark debe revivir el RDD y esto
#toma más tiempo
depteDF.rdd.cache()

MapPartitionsRDD[11] at javaToPython at NativeMethodAccessorImpl.java:0

In [12]:
#Verificar el tipo de persistencia
depteDF.rdd.getStorageLevel()

StorageLevel(False, True, False, False, 1)

#StorageLevel(False, True, False, False, 1) --> Esta salida lo que indica es
#la forma de persistencia 
#se recomienda que la replicación sea a partir de 3, ya que de esta forma la probabilidad de perder los datos es más baja


https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.StorageLevel.html#pyspark.StorageLevel

In [13]:
#Dar de baja a la persistencia del cache
depteDF.rdd.unpersist()

MapPartitionsRDD[11] at javaToPython at NativeMethodAccessorImpl.java:0

In [14]:
#Aplicar una persistencia
depteDF.rdd.persist(StorageLevel.MEMORY_AND_DISK_2)

MapPartitionsRDD[11] at javaToPython at NativeMethodAccessorImpl.java:0

In [15]:
#Verificar el tipo de persistencia
depteDF.rdd.getStorageLevel()

StorageLevel(True, True, False, False, 2)

In [16]:
#Crear el propio particionado
StorageLevel.MEMORY_AND_DISK_3 = StorageLevel(True, True, False, False, 3)

In [17]:
#Dar de baja a la persistencia del cache - 2
depteDF.rdd.unpersist()

MapPartitionsRDD[11] at javaToPython at NativeMethodAccessorImpl.java:0

In [18]:
#Aplicar una persistencia
depteDF.rdd.persist(StorageLevel.MEMORY_AND_DISK_3)

MapPartitionsRDD[11] at javaToPython at NativeMethodAccessorImpl.java:0

In [19]:
#Verificar el tipo de persistencia
depteDF.rdd.getStorageLevel()

StorageLevel(True, True, False, False, 3)

In [20]:
depteDF.rdd.unpersist()

MapPartitionsRDD[11] at javaToPython at NativeMethodAccessorImpl.java:0